# Benchmark FRESA.CAD BSWIMS final Script


This algorithm implementation uses R code and a Python library (rpy2) to connect with it, in order  to run the following it is necesary to have installed both on your computer:

- R (you can download in https://www.r-project.org/) <br>
- install rpy2 by <code> pip install rpy2 </code>

In [1]:
import numpy as np
import pandas as pd
import sys
from pathlib import Path
sys.path.append("../tadpole-algorithms")
import tadpole_algorithms
from tadpole_algorithms.models import Benchmark_FRESACAD_R
from tadpole_algorithms.preprocessing.split import split_test_train_tadpole
#rpy2 libs and funcs
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector
from rpy2.robjects import r, pandas2ri 
from rpy2 import robjects
from rpy2.robjects.conversion import localconverter


In [2]:
# Load D1_D2 train and possible test data set
data_path_train_test = Path("data/TADPOLE_D1_D2.csv")
data_df_train_test = pd.read_csv(data_path_train_test)

# Load data Dictionary
data_path_Dictionaty = Path("data/TADPOLE_D1_D2_Dict.csv")
data_Dictionaty = pd.read_csv(data_path_Dictionaty)

# Load D3 possible test set
data_path_test = Path("data/TADPOLE_D3.csv")
data_D3 = pd.read_csv(data_path_test)

# Load D4 evaluation data set 
data_path_eval = Path("data/TADPOLE_D4_corr.csv")
data_df_eval = pd.read_csv(data_path_eval)

# Split data in test, train and evaluation data
train_df, test_df, eval_df = split_test_train_tadpole(data_df_train_test, data_df_eval)

#instanciate the model to get the functions
model = Benchmark_FRESACAD_R()
#set the flag to true to use a preprocessed data
USE_PREPROC = False


c:\program files\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (471,473,474,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,569,570,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,599,601,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,624,625,626,627,628,629,630,631,632,633,634,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,72

In [3]:
#preprocess the data
D1Train,D2Test,D3Train,D3Test = model.extractTrainTestDataSets_R("data/TADPOLE_D1_D2.csv","data/TADPOLE_D3.csv")


2020-12-25 13:03:12,155 - MainThread - tadpole_algorithms.models.benchmark_FRESACAD_R - INFO - Extract Training and Testing sets


c:\program files\python38\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (1414,1657,1658,1660,1666,1667,1896,1903,1904,1905) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
c:\program files\python38\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (1658,1660,1666,1667,1896,1903,1905,1906) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
c:\program files\python38\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (1658,1660,1666,1667,1896,1904,1905) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [4]:
# AdjustedTrainFrame,testingFrame,Train_Imputed,Test_Imputed = model.preproc_tadpole_D1_D2(data_df_train_test,USE_PREPROC)
AdjustedTrainFrame,testingFrame,Train_Imputed,Test_Imputed = model.preproc_with_R(D1Train,D2Test,data_Dictionaty,usePreProc=USE_PREPROC)


2020-12-25 13:04:04,571 - MainThread - tadpole_algorithms.models.benchmark_FRESACAD_R - INFO - Prepocess Data Frames
2020-12-25 13:04:36,208 - MainThread - rpy2.rinterface_lib.callbacks - WARNING - R[write to console]: Loading required package: Rcpp

2020-12-25 13:04:36,675 - MainThread - rpy2.rinterface_lib.callbacks - WARNING - R[write to console]: Loading required package: stringr

2020-12-25 13:04:36,847 - MainThread - rpy2.rinterface_lib.callbacks - WARNING - R[write to console]: Loading required package: miscTools

2020-12-25 13:04:36,960 - MainThread - rpy2.rinterface_lib.callbacks - WARNING - R[write to console]: Loading required package: Hmisc

2020-12-25 13:04:36,968 - MainThread - rpy2.rinterface_lib.callbacks - WARNING - R[write to console]: Loading required package: lattice

2020-12-25 13:04:37,099 - MainThread - rpy2.rinterface_lib.callbacks - WARNING - R[write to console]: Loading required package: survival

2020-12-25 13:04:37,992 - MainThread - rpy2.rinterface_lib.call

In [5]:
#Train Congitive Models
modelfilename = model.Train_Congitive(AdjustedTrainFrame,usePreProc=USE_PREPROC)

2020-12-25 13:31:01,115 - MainThread - tadpole_algorithms.models.benchmark_FRESACAD_R - INFO - Train Cognitive Models
  [1] "AGE"                                             
  [2] "PTGENDER"                                        
  [3] "CDRSB"                                           
  [4] "ADAS11"                                          
  [5] "ADAS13"                                          
  [6] "MMSE"                                            
  [7] "RAVLT_learning"                                  
  [8] "RAVLT_immediate"                                 
  [9] "FAQ"                                             
 [10] "CDRSB_bl"                                        
 [11] "ADAS11_bl"                                       
 [12] "ADAS13_bl"                                       
 [13] "MMSE_bl"                                         
 [14] "RAVLT_learning_bl"                               
 [15] "RAVLT_immediate_bl"                              
 [16] "FAQ_bl"             

In [6]:
#Train ADAS/Ventricles Models
regresionModelfilename = model.Train_Regression(AdjustedTrainFrame,Train_Imputed,usePreProc=USE_PREPROC)
print(regresionModelfilename)

2020-12-25 14:12:56,719 - MainThread - tadpole_algorithms.models.benchmark_FRESACAD_R - INFO - Train ADAS13 and Ventricles Models
  [1] "AGE"                                             
  [2] "PTGENDER"                                        
  [3] "CDRSB"                                           
  [4] "ADAS11"                                          
  [5] "ADAS13"                                          
  [6] "MMSE"                                            
  [7] "RAVLT_learning"                                  
  [8] "RAVLT_immediate"                                 
  [9] "FAQ"                                             
 [10] "CDRSB_bl"                                        
 [11] "ADAS11_bl"                                       
 [12] "ADAS13_bl"                                       
 [13] "MMSE_bl"                                         
 [14] "RAVLT_learning_bl"                               
 [15] "RAVLT_immediate_bl"                              
 [16] "FAQ_bl" 

In [7]:
print(regresionModelfilename)
print(type(regresionModelfilename))

[1] "data/_RegressionModels_50_Nolog.RDATA"

<class 'rpy2.robjects.vectors.StrVector'>


In [8]:
#Predict 
Forecast_D2 = model.Forecast_All(modelfilename,
                                 regresionModelfilename,
                                 testingFrame,
                                 Test_Imputed,
                                 submissionTemplateFileName="data/TADPOLE_Simple_Submission_TeamName.xlsx",
                                 usePreProc=USE_PREPROC)

2020-12-25 16:01:36,503 - MainThread - tadpole_algorithms.models.benchmark_FRESACAD_R - INFO - Forecast Congitive Status, ADAS13 and Ventricles
2020-12-25 16:01:44,433 - MainThread - rpy2.rinterface_lib.callbacks - WARNING - R[write to console]: 
2020-12-25 16:01:44,434 - MainThread - rpy2.rinterface_lib.callbacks - WARNING - R[write to console]: -
2020-12-25 16:01:44,590 - MainThread - rpy2.rinterface_lib.callbacks - WARNING - R[write to console]: 
2020-12-25 16:01:44,591 - MainThread - rpy2.rinterface_lib.callbacks - WARNING - R[write to console]: \
                                                                               - R[write to console]: 
2020-12-25 16:01:44,592 - MainThread - rpy2.rinterface_lib.callbacks - WARNING - R[write to console]: 
 [1]   0   3   6  12  18  24  30  36  42  48  54  60  66  72  78  84  90  96 102
[20] 108 114 120
[1] 7660
[1] 896
[1] 896
[1] 0
[1] 7660
[1] 896
[1] 587
[1] 1
statusLO
  1   2   3 
342 341 212 
[1] 0.8648029 0.7377718 0.7631522
[1] 0.6

In [9]:
#data_forecast_test = Path("data/_ForecastFRESACAD.csv")
#Forecast_D2 = pd.read_csv(data_forecast_test)

from tadpole_algorithms.evaluation import evaluate_forecast
from tadpole_algorithms.evaluation import print_metrics
# Evaluate the model 
dictionary = evaluate_forecast(eval_df, Forecast_D2)
# Print metrics
print_metrics(dictionary)

c:\program files\python38\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=[0, 1, 2] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


[[76 10  0]
 [20 69  3]
 [ 2 10 20]]
mAUC (multiclass Area Under Curve): 0.892
bca (balanced classification accuracy): 0.816
adasMAE (ADAS13 Mean Absolute Error): 5.382
ventsMAE (Ventricles Mean Absolute Error), in % ICV: 0.454
adasWES (ADAS13 Weighted Error Score): 5.423
ventsWES (Ventricles Weighted Error Score ), in % ICV: 0.389
adasCPA (ADAS13 Coverage Probability Accuracy for 50% Confidence Interval: 0.331
ventsCPA (Ventricles Coverage Probability Accuracy for 50% Confidence Interval: 0.273


In [10]:
# AdjustedTrainFrame,testingFrame,Train_Imputed,Test_Imputed = model.preproc_tadpole_D1_D2(data_df_train_test,USE_PREPROC)
D3AdjustedTrainFrame,D3testingFrame,D3Train_Imputed,D3Test_Imputed = model.preproc_with_R(D3Train,
                                                                                    D3Test,
                                                                                    data_Dictionaty,
                                                                                    MinVisit=18,
                                                                                    colImputeThreshold=0.15,
                                                                                    rowImputeThreshold=0.10,
                                                                                    includeID=False,
                                                                                    usePreProc=USE_PREPROC)

2020-12-25 16:27:51,148 - MainThread - tadpole_algorithms.models.benchmark_FRESACAD_R - INFO - Prepocess Data Frames
[1] "Cortical Thickness Average of RightBankssts"               
[2] "Cortical Thickness Average of RightCaudalAnteriorCingulate"
[3] "Cortical Thickness Average of RightCaudalMiddleFrontal"    
[4] "Cortical Thickness Average of RightBankssts"               
[5] "Cortical Thickness Average of RightCaudalAnteriorCingulate"
[1] "Cortical Thickness Average of LeftBankssts"               
[2] "Cortical Thickness Average of LeftCaudalAnteriorCingulate"
[3] "Cortical Thickness Average of LeftCaudalMiddleFrontal"    
[4] "Cortical Thickness Average of LeftBankssts"               
[5] "Cortical Thickness Average of LeftCaudalAnteriorCingulate"
[1] 383 782
  [1] "D1"                                            
  [2] "D2"                                            
  [3] "SITE"                                          
  [4] "ORIGPROT"                                      
  [5] 

In [11]:
#Train Congitive Models
D3modelfilename = model.Train_Congitive(D3AdjustedTrainFrame,usePreProc=USE_PREPROC)

2020-12-25 16:31:47,407 - MainThread - tadpole_algorithms.models.benchmark_FRESACAD_R - INFO - Train Cognitive Models
  [1] "AGE"                                             
  [2] "PTGENDER"                                        
  [3] "CDRSB"                                           
  [4] "ADAS11"                                          
  [5] "ADAS13"                                          
  [6] "MMSE"                                            
  [7] "RAVLT_learning"                                  
  [8] "RAVLT_immediate"                                 
  [9] "FAQ"                                             
 [10] "CDRSB_bl"                                        
 [11] "ADAS11_bl"                                       
 [12] "ADAS13_bl"                                       
 [13] "MMSE_bl"                                         
 [14] "RAVLT_learning_bl"                               
 [15] "RAVLT_immediate_bl"                              
 [16] "FAQ_bl"             

In [12]:
#Train ADAS/Ventricles Models
D3regresionModelfilename = model.Train_Regression(D3AdjustedTrainFrame,D3Train_Imputed,usePreProc=USE_PREPROC)


2020-12-25 16:36:07,635 - MainThread - tadpole_algorithms.models.benchmark_FRESACAD_R - INFO - Train ADAS13 and Ventricles Models
  [1] "AGE"                                             
  [2] "PTGENDER"                                        
  [3] "CDRSB"                                           
  [4] "ADAS11"                                          
  [5] "ADAS13"                                          
  [6] "MMSE"                                            
  [7] "RAVLT_learning"                                  
  [8] "RAVLT_immediate"                                 
  [9] "FAQ"                                             
 [10] "CDRSB_bl"                                        
 [11] "ADAS11_bl"                                       
 [12] "ADAS13_bl"                                       
 [13] "MMSE_bl"                                         
 [14] "RAVLT_learning_bl"                               
 [15] "RAVLT_immediate_bl"                              
 [16] "FAQ_bl" 

In [13]:
#Predict 
Forecast_D3 = model.Forecast_All(D3modelfilename,
                                 D3regresionModelfilename,
                                 D3testingFrame,
                                 D3Test_Imputed,
                                 submissionTemplateFileName="data/TADPOLE_Simple_Submission_TeamName.xlsx",
                                 usePreProc=USE_PREPROC)

2020-12-25 17:11:27,199 - MainThread - tadpole_algorithms.models.benchmark_FRESACAD_R - INFO - Forecast Congitive Status, ADAS13 and Ventricles
2020-12-25 17:11:28,707 - MainThread - rpy2.rinterface_lib.callbacks - WARNING - R[write to console]: 
2020-12-25 17:11:28,708 - MainThread - rpy2.rinterface_lib.callbacks - WARNING - R[write to console]: -
2020-12-25 17:11:28,847 - MainThread - rpy2.rinterface_lib.callbacks - WARNING - R[write to console]: 
2020-12-25 17:11:28,848 - MainThread - rpy2.rinterface_lib.callbacks - WARNING - R[write to console]: \
                                                                               - R[write to console]: 
2020-12-25 17:11:28,848 - MainThread - rpy2.rinterface_lib.callbacks - WARNING - R[write to console]: 
numeric(0)
[1] 896
[1] 896
[1] 896
[1] 896
[1] 896
[1] 10521
[1] 896
[1] 10104
[1] 192
statusLO
  1   2   3 
299 269 136 
[1] 0.9083706 0.8237333 0.9530631
[1] 0.8490431 0.2647059 0.1200929
[1] 0.8167413 0.6474667 0.9061261
 [1] 25.9185

In [14]:
from tadpole_algorithms.evaluation import evaluate_forecast
from tadpole_algorithms.evaluation import print_metrics
# Evaluate the D3 model 
dictionary = evaluate_forecast(eval_df,Forecast_D3)

# Print metrics
print_metrics(dictionary)

[[74 11  1]
 [20 62 10]
 [ 2 11 19]]
mAUC (multiclass Area Under Curve): 0.858
bca (balanced classification accuracy): 0.784
adasMAE (ADAS13 Mean Absolute Error): 5.631
ventsMAE (Ventricles Mean Absolute Error), in % ICV: 1.110
adasWES (ADAS13 Weighted Error Score): 5.659
ventsWES (Ventricles Weighted Error Score ), in % ICV: 1.111
adasCPA (ADAS13 Coverage Probability Accuracy for 50% Confidence Interval: 0.162
ventsCPA (Ventricles Coverage Probability Accuracy for 50% Confidence Interval: 0.393


c:\program files\python38\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=[0, 1, 2] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
